In [1]:
#%load_ext autoreload
#%autoreload 2
%matplotlib inline
%cd '../'

/Users/laurentperrinet/quantic/science/HomeHots/HOTS_clone_laurent


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
import time
# timestr = '20201021'
import datetime
# https://en.wikipedia.org/wiki/ISO_8601
timestr = datetime.datetime.now().date().isoformat()

### Building matrix for logistic regression

In [3]:
%ls -ltrh /tmp/2*

-rw-r--r--  1 laurentperrinet  wheel    67M Nov  5 15:00 /tmp/20201021_hots_event0_o-sub.p
-rw-r--r--  1 laurentperrinet  wheel   187M Nov  5 15:24 /tmp/20201021_hots_event0_o.p
-rw-r--r--  1 laurentperrinet  wheel   351M Nov  5 15:26 /tmp/20201021_hots_vectorzLR_subsample_X.npy
-rw-r--r--  1 laurentperrinet  wheel    78K Nov  5 15:26 /tmp/20201021_hots_vectorzLR_subsample_y.npy
-rw-r--r--  1 laurentperrinet  wheel   2.8G Nov  5 15:28 /tmp/20201021_hots_vectorzLR-full_X.npy
-rw-r--r--  1 laurentperrinet  wheel   161K Nov  5 15:28 /tmp/20201021_hots_vectorzLR-full_y.npy


In [4]:
tau = 9e-4 # -> tau=1ms, si on prend 10 ms on est à 1s pour la dernière couche et les vidéos font 0.3s en moyenne
R = 2
filthr = 2
nbkNN = 3
algo = 'lagorce'
decay = 'exponential'
hom = True
krnlinit = 'rdn'
nb_cluster = [4, 8, 16]
ImageSize = (34, 34)
DataPath = 'Data/testsetnmnist.p'

NbClusteringData = 2500
NbTrainingData = 5000
NbTestingData = 1500

#timestr = time.strftime("%Y%m%d")
#fname = 'Records/EXP_03_NMNIST/'+timestr+'_hots_vectorzLR.pkl'
fname = '/tmp/'+timestr+'_hots_vectorzLR-full'

In [ ]:
c_int = lambda n, d : ((n - 1) // d) + 1

for hom in [False, True]:

    if hom==True:
        fname_model = 'Records/EXP_03_NMNIST/'+timestr+'_hots_'+str(tau*1000)+'ms_'+algo+'_homeo.pkl'
        fname_event0_o = 'Records/EXP_03_NMNIST/'+timestr+'_hots_event0_o_homeo.p'
    else:
        fname_model = 'Records/EXP_03_NMNIST/'+timestr+'_hots_'+str(tau*1000)+'ms_'+algo+'.pkl'
        fname_event0_o = 'Records/EXP_03_NMNIST/'+timestr+'_hots_event0_o.p'

    print(fname_model)
    
    import pickle
    from HOTS.ToolsMonitor import GenerateActivationMap, DisplayActivationMap
    from HOTS.Event import Event, SimpleAlphabet, LoadNMNIST
    from HOTS.STS import STS
    from HOTS.Layer import ClusteringLayer
    from HOTS.ToolsMonitor import (
        DisplayImage,
        DisplaySurface3D,
        DisplaySurface2D,
        DisplayConvergence,
    )
    from HOTS.Tools import SaveObject, LoadObject
    from HOTS.Network import Network
    from HOTS.Event import conv2eve

    event_tr, event_te, event_cl, label_tr, label_te = LoadNMNIST(
    NbTrainingData, NbTestingData, NbClusteringData, OneOfEach=False, Path=DataPath, OutOnePolarity=False, ListPolarities=None, verbose=0)

    L1 = ClusteringLayer(tau=tau,R=R,verbose=0,ThrFilter=filthr,LearningAlgo=algo,kernel=decay,homeo=hom,init=krnlinit)
    L2 = ClusteringLayer(tau=10 * tau,R=2 * R,verbose=0,ThrFilter=filthr,LearningAlgo=algo,kernel=decay,homeo=hom,init=krnlinit)
    L3 = ClusteringLayer(tau=10 * 10 * tau,R=2 * 2 * R,verbose=0,ThrFilter=filthr,LearningAlgo=algo,kernel=decay,homeo=hom,init=krnlinit)
    Net = Network([L1, L2, L3])    
    
    if not os.path.isfile(fname_model):
        ClusterLayer, event_output = Net.TrainCluster(
                event=event_cl, NbClusterList=nb_cluster, to_record=True, NbCycle=1
            )
        SaveObject(ClusterLayer, fname_model)
    else: 
        ClusterLayer, Classif0 = LoadObject(fname_model)

    if not os.path.isfile(fname_event0_o):            
        print('run the events through the network')
        homrun = False
        event0_o_tr2 = Net.RunNetwork(event_tr, NbClusterList=ClusterLayer, homrun=homrun)
        event0_o_te2 = Net.RunNetwork(event_te, NbClusterList=ClusterLayer, homrun=homrun)
        SaveObject([event0_o_tr2, event0_o_te2], fname_event0_o)
    else: 
        event0_o_tr2, event0_o_te2 = LoadObject(fname_event0_o)
    

Records/EXP_03_NMNIST/2020-11-05_hots_0.9ms_lagorce.pkl
[]


In [ ]:
def gather_data(events, tau_cla = .300, # characteristic time of a digit 
                sample_events = 200,    sample_space = 1):
    n_events = events.time.shape[0]

    data = np.zeros((c_int(events.ImageSize[0], sample_space), c_int(events.ImageSize[1], sample_space), len(events.ListPolarities))) #tmp data


    data_len = len(data.ravel())
    X = np.zeros((c_int(n_events, sample_events), data_len))
    y = np.zeros((c_int(n_events, sample_events), ))

    t_absolute = 0
    t_old = 0
    i_label_tr = 0
    for i in range(1, n_events-1):
        if i == events.ChangeIdx[i_label_tr]:
            i_label_tr += 1
            t_absolute += events.time[i]

        t_new = t_absolute + events.time[i]
        data *= np.exp(-(t_new-t_old)/tau_cla)
        t_old = t_new

        data[events.address[i, 0]//sample_space, 
             events.address[i,1]//sample_space, 
             events.polarity[i]] = 1.

        if i % sample_events == 0:
            # print(f'{i=} {n_events=} {n_events//sample_events=}')
            X[i//sample_events, :] = data.ravel()
            y[i//sample_events] = label_tr[i_label_tr]

    print('Number of events: '+str(X.shape[0])+' - Number of features: '+str(X.shape[1]))

    return X, y


### Performing logistic regression with cross-validation

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegressionCV.html


In [ ]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split


def classify(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

    lr = LogisticRegressionCV(random_state=0, max_iter=10000).fit(X_train, y_train)

    return lr.score(X_test, y_test)

for sample_space in [4, 2, 1]:
    X, y = gather_data(event0_o_tr2, sample_space=sample_space)
    print(f'Classification score for {sample_space=} is {classify(X, y):.3f}')

In [ ]:
for homeo in [False, True]:
    if homeo:
        fname_event0_o = 'Records/EXP_03_NMNIST/'+timestr+'_hots_event0_o_homeo.p'
    else:
        fname_event0_o = 'Records/EXP_03_NMNIST/'+timestr+'_hots_event0_o.p'
    event0_o_tr2, event0_o_te2 = LoadObject(fname_event0_o)

    X_train, y_train = gather_data(event0_o_tr2)
    lr = LogisticRegressionCV(random_state=0, max_iter=10000).fit(X_train, y_train)
    X_test, y_test = gather_data(event0_o_te2)
    print(f'Classification score for {homeo=} is {lr.score(X_test, y_test):.3f}')    


    for tau_cla in .3 * np.logspace(-1, 1, 7, base=4):
        X_train, y_train = gather_data(event0_o_tr2, tau_cla=tau_cla)
        lr = LogisticRegressionCV(random_state=0, max_iter=10000).fit(X_train, y_train)
        X_test, y_test = gather_data(event0_o_te2, tau_cla=tau_cla)
        print(f'Classification score for {tau_cla=} is {lr.score(X_test, y_test):.3f}')

    for sample_space in [4, 2, 1]:
        X_train, y_train = gather_data(event0_o_tr2, sample_space=sample_space)
        lr = LogisticRegressionCV(random_state=0, max_iter=10000).fit(X_train, y_train)
        X_test, y_test = gather_data(event0_o_te2, sample_space=sample_space)
        print(f'Classification score for {sample_space=} is {lr.score(X_test, y_test):.3f}')


    for sample_events in [1000, 500, 200, 100, 50]:
        X_train, y_train = gather_data(event0_o_tr2, sample_events=sample_events)
        lr = LogisticRegressionCV(random_state=0, max_iter=10000).fit(X_train, y_train)
        X_test, y_test = gather_data(event0_o_te2, sample_events=sample_events)
        print(f'Classification score for {sample_events=} is {lr.score(X_test, y_test):.3f}')